In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
num_cols = ['Qж ТМ, м3/сут', 'Qн, т/сут', 'Qн ТМ, т/сут',
                        'ГФ, м3/т', 'ГФ ТМ, м3/т', 'Обв ХАЛ, %', 'Нд, м',
                        'Рзатр, атм', 'Dшт, мм', 'КВЧ', 'I, А', 'F, Гц',
                        'Рприем, атм', 'Мехпримеси (ХАЛ), мг/дм3']

In [3]:
from tools import pre_process_only_bad
from tools import pre_process_normal

In [ ]:
break_data = pre_process_only_bad(all_paths=pd.read_csv('cooked data\\_squajins.csv')['0'].values,
                        source='cooked data\\',
                        num_cols=num_cols)
###break_data.to_csv("break_data.csv")

In [114]:
from collections import Counter

In [115]:
Counter(break_data['Скважина'])

Counter({'309': 7,
         '1025': 14,
         '1023': 14,
         '1031': 14,
         '1008': 28,
         '927': 7,
         '624': 28,
         '1020': 7,
         '1026': 14,
         '345': 28,
         '331': 7,
         '335': 7,
         '346': 7,
         '634': 28,
         '947': 7,
         '628': 21,
         '946': 7,
         '1032': 14,
         '1030': 7,
         '948': 21,
         '953': 14,
         '449': 7,
         '731': 7,
         '452': 14,
         '451': 14,
         '937': 14,
         '936': 14,
         '671': 28,
         '1103': 14,
         '1112': 7,
         '1047': 7,
         '1113': 7,
         '361': 7,
         '362': 14,
         '372': 21,
         '1040': 14,
         '137': 7,
         '371': 14,
         '636': 7,
         '635': 7,
         '633': 14,
         '976': 28,
         '977': 14,
         '968': 7,
         '964': 14,
         '957': 14,
         '106': 14,
         '124': 7,
         '301': 7,
         '304': 14,
        

In [86]:
#break_data = pd.read_csv("break_data.csv")

In [ ]:
reduced, o = pre_process_normal(all_paths=pd.read_csv('cooked data\\_squajins.csv')['0'].values,
                        source='cooked data\\',
                        num_cols=num_cols,
                        filter_method='delete')
#reduced.to_csv('reduced.csv')

In [ ]:
#reduced =  pd.read_csv("reduced.csv")

In [107]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def generate_datasets(data, window_size,scale=True, scaler_type=StandardScaler):
    _l = len(data) 
    data = scaler_type().fit_transform(data)
    Xs = []
    Ys = []
    for i in range(0, (_l - window_size)):
        Xs.append(data[i:i+window_size] + np.random.normal(loc=0.0,
                                                           scale=0.1,
                                                           size=data[i:i+window_size].shape)) 
        Ys.append(data[i:i+window_size]) 

    return  Xs, Ys

In [35]:
from keras import metrics
import keras
import tensorflow as tf
import os

In [73]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import mean_squared_error

In [74]:
def perf_measure(true, pred):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    true = list(true)
    pred = list(pred)
    
    for i in range(len(true)): 
        if pred[i]==1 and true[i]==pred[i]:
            TP += 1
        if pred[i]==1 and true[i]!=pred[i]:
            FP += 1
        if pred[i]==0 and true[i]==pred[i]:
            TN += 1
        if pred[i]==0 and true[i]!=pred[i]:
            FN += 1

    return(TP, FP, TN, FN)

def evaluate(true, pred):
    print('accuracy_score: ' + str(accuracy_score(true, pred)))
    print('precision_score: ' + str(precision_score(true, pred)))
    print('recall_score: '+ str(recall_score(true, pred)))
    TP, FP, TN, FN = perf_measure(pred, true)
    print(perf_measure(true, pred))
    print('TruePositive: ' + str(TP) + ' TrueNegative: ' + str(TN))
    print('FalsePositive: ' + str(FP) + ' FalseNegative: ' + str(FN))

In [108]:
epochs = 100
batch_size = 32
window_length = 7
feats = 14

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='loss', min_delta=1e-2, patience=5, verbose=0, mode='auto',
    baseline=None, restore_best_weights=True)

model = keras.Sequential()
model.add(keras.layers.LSTM(64, kernel_initializer='he_uniform', batch_input_shape=(None, window_length, feats), return_sequences=True, name='encoder_1'))
model.add(keras.layers.LSTM(32, kernel_initializer='he_uniform', return_sequences=True, name='encoder_2'))
model.add(keras.layers.LSTM(16, kernel_initializer='he_uniform', return_sequences=False, name='encoder_3'))
model.add(keras.layers.RepeatVector(window_length, name='encoder_decoder_bridge'))
model.add(keras.layers.LSTM(16, kernel_initializer='he_uniform', return_sequences=True, name='decoder_1'))
model.add(keras.layers.LSTM(32, kernel_initializer='he_uniform', return_sequences=True, name='decoder_2'))
model.add(keras.layers.LSTM(64, kernel_initializer='he_uniform', return_sequences=True, name='decoder_3'))
model.add(keras.layers.TimeDistributed(keras.layers.Dense(feats)))
model.compile(loss="mse",optimizer='adam')
model.build()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_1 (LSTM)             (None, 7, 64)             20224     
_________________________________________________________________
encoder_2 (LSTM)             (None, 7, 32)             12416     
_________________________________________________________________
encoder_3 (LSTM)             (None, 16)                3136      
_________________________________________________________________
encoder_decoder_bridge (Repe (None, 7, 16)             0         
_________________________________________________________________
decoder_1 (LSTM)             (None, 7, 16)             2112      
_________________________________________________________________
decoder_2 (LSTM)             (None, 7, 32)             6272      
_________________________________________________________________
decoder_3 (LSTM)             (None, 7, 64)             2

In [ ]:
sqerr1 = []
sqerr2 = []
persitions = []
accuracys = []
recalls = []

for sq in set(reduced['Скважина']):
    curr_test = reduced[reduced['Скважина'] == sq]
    curr_train = reduced[reduced['Скважина'] != sq][:10000]
    
    break_test = break_data[break_data['Скважина'] != sq]
    
    _, clean_test = generate_datasets(curr_test[num_cols], 7)
    noisy_train, clean_train = generate_datasets(curr_train[num_cols], 7)
    _, clean_break = generate_datasets(break_test[num_cols], 7)
    
    print(len(clean_test), len(clean_break))
    if len(clean_test) > len(clean_break):
        clean_test = clean_test[:len(clean_break)]
    else:
        clean_break = clean_break[:len(clean_test)]
        
    if len(clean_test) < 21:
        continue
    
    print('=========== model', sq, "fitting =====================")
    model.fit(np.stack(noisy_train), np.stack(clean_train), epochs=25, batch_size=32, verbose=0)
  
    predictions = []
    true = []

    summ = 0
    predicted = model.predict(np.stack(clean_break))
    for i in range(len(clean_break)):
        true.append(1)
        mse = mean_squared_error(clean_break[i], predicted[i])
        summ += mse
        if mse > 1.47:
            predictions.append(1)
        else:
            predictions.append(0)
            
    res = summ / len(clean_break)
    sqerr2.append(res)
    print("mse break", res)
    
    summ = 0
    predicted = model.predict(np.stack(clean_test))
    for i in range(len(clean_test)):
        true.append(0)
        mse = mean_squared_error(clean_test[i], predicted[i])
        summ += mse
        if mse > 1.47:
            predictions.append(1)
        else:
            predictions.append(0)
            
    res = summ / len(clean_test)
    sqerr1.append(res)
    print("mse normal (clean)", res)  
    
    evaluate(true, predictions)
    
    keras.backend.clear_session()    

210 763
=========== model 467 fitting =====================
